In [1]:
from data_loader import *
from __future__ import division
from __future__ import print_function
import pandas as pd
import bokeh
import numpy as np

import bokeh.plotting as bp
from bokeh.plotting import figure, output_file, show,gridplot
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.axes import DatetimeAxis

from datetime import datetime
from datetime import timedelta

import pandas as pd

bp.output_notebook()

Loading BokehJS ...

In [2]:
data = DataLoader('E3_7E_44_77_F2_38.scn') #badge data from short experiment
#data = DataLoader('FF_6B_D4_CB_22_FA.scn') #badge data from short experiment

session_start_time = datetime(2016,1,27,14,26,0)
session_end_time = datetime(2016,1,27,14,46,0)

#session_start_time = datetime(2016,1,27,15,10,0)
#session_end_time = datetime(2016,1,27,16,5,0)

#data = DataLoader('badge_v33_EC_21_82_A8_0B_59.scn') #badge data from a 4 days test

In [3]:
freq = data.data[0]['Frequency'] #assuming same freq. for all samples

In [4]:
# take only samples
signals = map(lambda x: x['samples'], data.data)

# convert 
con_signals = [map(lambda x: {"datetime": x["time"], "signal": x['signal']}, i)  for i in signals]
#con_signals[0]

# Append all samples into an array
singals_all = []
for s in con_signals:
    singals_all.extend(s)
    
# sort
sdata = sorted(singals_all, key=lambda x: x['datetime'])

# Check for gaps in data (before loading into pandas. It's faster)

In [5]:
def check_raw_gaps(arr, max_delta_sec = 15):
    #max_diff = np.timedelta64(max_delta_sec, 's')
    max_diff = timedelta(seconds=1);
    prev = None
    for i in range (0,len(arr)): # iterate over all lines
        #print df.iloc[i].datetime
        curr = arr[i]['datetime']
        if (prev):
            diff = curr-prev
            if diff > max_diff:
                print ("{},{},{}".format(curr,prev,diff))
                #break
        prev = curr

In [6]:
check_raw_gaps(sdata,15)

2016-01-27 12:13:06,2016-01-26 23:34:53.750000,12:38:12.250000
2016-01-27 14:21:02,2016-01-27 14:08:07.750000,0:12:54.250000
2016-01-27 16:59:49,2016-01-27 16:58:06.750000,0:01:42.250000


# Loading data infor Pandas

In [7]:
df = pd.DataFrame()

In [8]:
sdata[10:20]

[{'datetime': datetime.datetime(2016, 1, 26, 23, 33, 29, 500000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 29, 750000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 30), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 30, 250000), 'signal': 2},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 30, 500000), 'signal': 2},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 30, 750000), 'signal': 2},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 31), 'signal': 2},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 31, 250000), 'signal': 2},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 31, 500000), 'signal': 1},
 {'datetime': datetime.datetime(2016, 1, 26, 23, 33, 31, 750000), 'signal': 2}]

In [9]:
# Append all samples into a DataFrame
#for s in sdata:
df = df.append(pd.DataFrame(sdata))

In [10]:
df = df.set_index(pd.DatetimeIndex(df['datetime']))
#df.index

In [11]:
df_fil = df.ix[session_start_time:session_end_time]

In [12]:
df_fil[0:10]

,datetime,signal
2016-01-27 14:26:00.000,2016-01-27 14:26:00.000,2
2016-01-27 14:26:00.250,2016-01-27 14:26:00.250,2
2016-01-27 14:26:00.500,2016-01-27 14:26:00.500,2
2016-01-27 14:26:00.750,2016-01-27 14:26:00.750,2
2016-01-27 14:26:01.000,2016-01-27 14:26:01.000,2
2016-01-27 14:26:01.250,2016-01-27 14:26:01.250,3
2016-01-27 14:26:01.500,2016-01-27 14:26:01.500,2
2016-01-27 14:26:01.750,2016-01-27 14:26:01.750,2
2016-01-27 14:26:02.000,2016-01-27 14:26:02.000,2
2016-01-27 14:26:02.250,2016-01-27 14:26:02.250,2


In [13]:
# calc a moving average
sr_mean = pd.rolling_mean(df_fil['signal'], window=4*60, min_periods=1, center=False) # 4 samples per sec, 60 seconds

# smooth signal
sr_smooth = pd.rolling_mean(df_fil['signal'], window=5, min_periods=1, center=False) # 5 samples

sr_mean[0:10]

2016-01-27 14:26:00.000    2.000000
2016-01-27 14:26:00.250    2.000000
2016-01-27 14:26:00.500    2.000000
2016-01-27 14:26:00.750    2.000000
2016-01-27 14:26:01.000    2.000000
2016-01-27 14:26:01.250    2.166667
2016-01-27 14:26:01.500    2.142857
2016-01-27 14:26:01.750    2.125000
2016-01-27 14:26:02.000    2.111111
2016-01-27 14:26:02.250    2.100000
Name: signal, dtype: float64

In [14]:
# calc RMS
smpls = 4 * 30;
#series_sq=df_fil['signal']*df_fil['signal'] # sqaure values
series_sq=sr_smooth*sr_smooth # sqaure values
series_sq_mean = pd.rolling_mean(series_sq, window=smpls, min_periods=1, center=False) # mean square
series_sq_rms = series_sq_mean.apply(np.sqrt)

In [15]:
tools = "xwheel_zoom,xpan,reset,resize,save"
p = figure(title="Signal", x_axis_label='x', y_axis_label='y'
           ,x_axis_type="datetime",width=1000, height=400,y_range=[0, 20], tools=tools)
# add a line renderer with legend and line thickness
p.line(df_fil.index.to_pydatetime(), df_fil.signal, legend="Raw", line_width=2, color='blue')
p.line(sr_smooth.index,sr_smooth, legend="Smooth5", line_width=2, color='Red')
p.line(series_sq_rms.index,series_sq_rms, legend="RMS30", line_width=2, color='Gray')
p.line(sr_mean.index,sr_mean, legend="Mean60", line_width=2, color='green')
p.below[0].formatter.formats = dict(years=['%Y'],months=['%b %Y'],days=['%d %b %Y'],hours=["%I:%M %P"],minutes=["%I:%M %P"]) #minutes is the one i'm interested in

# show the results
show(p)

In [16]:
#
#series_sq_rms
df_full = df_fil.copy()
df_full['smooth']=sr_smooth
df_full['rms']=series_sq_rms

# is speaking interval
df_full['is_speak'] = df_full.apply(lambda x : 1 if x['smooth'] > x['rms'] else 0, axis=1)

In [17]:
df_full

,datetime,signal,smooth,rms,is_speak
2016-01-27 14:26:00.000,2016-01-27 14:26:00.000,2,2.0,2.000000,0
2016-01-27 14:26:00.250,2016-01-27 14:26:00.250,2,2.0,2.000000,0
2016-01-27 14:26:00.500,2016-01-27 14:26:00.500,2,2.0,2.000000,0
2016-01-27 14:26:00.750,2016-01-27 14:26:00.750,2,2.0,2.000000,0
2016-01-27 14:26:01.000,2016-01-27 14:26:01.000,2,2.0,2.000000,0
2016-01-27 14:26:01.250,2016-01-27 14:26:01.250,3,2.2,2.034699,1
2016-01-27 14:26:01.500,2016-01-27 14:26:01.500,2,2.2,2.059126,1
2016-01-27 14:26:01.750,2016-01-27 14:26:01.750,2,2.2,2.077258,1
2016-01-27 14:26:02.000,2016-01-27 14:26:02.000,2,2.2,2.091252,1
2016-01-27 14:26:02.250,2016-01-27 14:26:02.250,2,2.2,2.102380,1


In [18]:
tools = "xwheel_zoom,xpan,reset,resize,save"
p1 = figure(title="Signal", x_axis_label='x', y_axis_label='y'
           ,x_axis_type="datetime",width=1000, height=400,y_range=[0, 20], tools=tools)
# add a line renderer with legend and line thickness
p1.line(df_full.index,df_full['is_speak'], legend="Speak", line_width=2, color='blue')
p1.line(df_full.index,df_full['smooth'], legend="Smooth5", line_width=2, color='Red')
p1.line(df_full.index,df_full['rms'], legend="RMS30", line_width=2, color='Gray')
p1.below[0].formatter.formats = dict(years=['%Y'],months=['%b %Y'],days=['%d %b %Y'],hours=["%I:%M %P"],minutes=["%I:%M %P"]) #minutes is the one i'm interested in

# show the results
show(p1)

In [19]:
# remove short intervals, "fill in" short blanks
# expects a data frame that indicates start time of samples that include speak. The freq indicates the time
# between samples (so each sample actually represents a speaking interval which is freq ms long).
def determine_speakers_intervals(df_speaker, group_max_time_diff_ms = 1000, min_interval_time_ms = 300, freq_ms = 250):
        def groupwhile(df, fwhile):
            groups = []
            i = 0
            while i < len(df):
                j = i
                while j < len(df) - 1 and fwhile(i,j, j + 1):
                    j = j + 1
                groups.append(df.iloc[i:j + 1])
                i = j + 1
            return groups

        #group_max_time_diff_sec = max md between records from the same user
        #min_interval_time_ms = minimum ms duration of speaking interval
        def get_speakers_intervals(df,group_max_time_diff_ms,min_interval_time_ms,freq_ms):
                group_max_time_diff = np.timedelta64(group_max_time_diff_ms, 'ms')
                min_interval_time = np.timedelta64(min_interval_time_ms, 'ms')
                np_freq = np.timedelta64(freq_ms, 'ms')
                groups = groupwhile(df, lambda start,curr, next: df.index[next]-df.index[curr] <= group_max_time_diff)
                values = []
                for g in groups:
                    values.append([g.index[0], g.index[-1]+np_freq,len(g.index)]) # end time shoud include the end of the time interval
                stops = pd.DataFrame(values, columns=['start_time', 'end_time','cnt'])
                stops = stops[stops.end_time - stops.start_time >= min_interval_time]
                return stops

        speakers_intervals = get_speakers_intervals(df_speaker,group_max_time_diff_ms,min_interval_time_ms,freq_ms) # allow 3 timeslots of silence, interval should be at least 500 ms long
        return speakers_intervals

In [23]:
df_speak_only = df_full[df_full.is_speak == 1]
speak_intervals = determine_speakers_intervals(df_speaker=df_speak_only,group_max_time_diff_ms=1000, min_interval_time_ms = 300,freq_ms=freq)

In [29]:
speak_intervals[0:100]

,start_time,end_time,cnt
0,2016-01-27 14:26:01.250,2016-01-27 14:26:02.500,5
1,2016-01-27 14:26:06.750,2016-01-27 14:26:11.000,17
2,2016-01-27 14:26:13.500,2016-01-27 14:26:15.000,6
3,2016-01-27 14:26:16.250,2016-01-27 14:26:26.250,35
4,2016-01-27 14:26:28.250,2016-01-27 14:26:29.500,5
5,2016-01-27 14:26:30.750,2016-01-27 14:26:36.750,22
7,2016-01-27 14:26:48.750,2016-01-27 14:26:50.000,5
8,2016-01-27 14:26:53.500,2016-01-27 14:26:54.750,5
9,2016-01-27 14:26:57.500,2016-01-27 14:27:03.000,16
10,2016-01-27 14:27:08.500,2016-01-27 14:27:10.750,9


In [21]:
p2 = figure(title="Speaking intervals", y_axis_label='Speaker',x_axis_type="datetime",plot_width=1000,plot_height=300
            ,x_range=p1.x_range, tools=tools)#, y_range=p1.y_range)
x = []
y = []
x2 = []
y2 = []

for i in range(len(speak_intervals)):
    #print speakers_intervals.iloc[i]
    r = speak_intervals.iloc[i]
    x.append(r.start_time)
    y.append(1)
    x2.append(r.end_time)
    y2.append(1)

p2.segment(x, y, x2, y2,  
        line_width=15, line_alpha=1)

p = gridplot([[p1],[p2]])
show(p)

# export data

In [22]:
df_fil.to_csv("E3_7E_44_77_F2_38_fil.csv", sep=',',index=False)

In [28]:
df_full.to_csv("E3_7E_44_77_F2_38_fil_full.csv", sep=',',index=False)